In [3]:
def account_acc_hier(filename_suffix):
    acc_matrix = []
    for i in range(5):
        concGothr = []
        for j in range(2):
            with open('./'+filename_suffix+str(i)+str(j)+'.out','r') as f:
                best_val_acc = 0.0
                last_test_acc = 0.0
                test_record_flag = False
                info = f.readlines()
                for line in info:
                    if line.startswith('val set accuracy:'):
                        tokens = line.split()
                        val_acc = float(tokens[-2])
                        if val_acc >= best_val_acc:
                            best_val_acc = val_acc
                            test_record_flag = True

                    if line.startswith('test set accuracy:'):
                        tokens = line.split()
                        test_acc = float(tokens[-2])
                        if test_record_flag:
                            last_test_acc = test_acc
                            test_record_flag = False

            concGothr.append((best_val_acc, last_test_acc))
        acc_matrix.append(concGothr)
    return acc_matrix

def account_acc(filename_suffix):
    acc_matrix = []
    for i in range(5):
        with open('./'+filename_suffix+str(i)+'.out','r') as f:
            best_val_acc = 0.0
            last_test_acc = 0.0
            test_record_flag = False
            info = f.readlines()
            for line in info:
                if line.startswith('val set accuracy:'):
                    tokens = line.split()
                    val_acc = float(tokens[-2])
                    if val_acc >= best_val_acc:
                        best_val_acc = val_acc
                        test_record_flag = True

                if line.startswith('test set accuracy:'):
                    tokens = line.split()
                    test_acc = float(tokens[-2])
                    if test_record_flag:
                        last_test_acc = test_acc
                        test_record_flag = False

            acc_matrix.append((best_val_acc, last_test_acc))
    return acc_matrix


In [23]:
atthier_acc = account_acc_hier('ATTCVHier')

atthier_acc[1][1] = (0.0, 0.0)
atthier_acc[2][0] = (0.0, 0.0)

atthier_acc

[[(81.6711590296, 70.6039551042), (80.0539083558, 71.0849812934)],
 [(80.5929919137, 72.528059861), (0.0, 0.0)],
 [(0.0, 0.0), (82.4797843666, 70.5505077499)],
 [(80.0539083558, 73.4366648851), (79.5148247978, 72.1004810262)],
 [(77.358490566, 70.4970603955), (78.4366576819, 71.9935863175)]]

In [4]:
nonatthier_acc = account_acc_hier('NonAttHier')
nonatthier_acc

[[(81.6711590296, 72.4211651523), (80.3234501348, 70.2298236237)],
 [(80.5929919137, 71.2453233565), (82.2102425876, 71.1918760021)],
 [(81.4016172507, 72.528059861), (81.4016172507, 70.1763762694)],
 [(80.8625336927, 69.214323891), (81.9407008086, 74.7194013896)],
 [(77.0889487871, 72.8487439872), (76.5498652291, 69.6953500802)]]

In [24]:
import numpy as np

ave_concate_hier = np.sum(np.asarray([atthier_acc[i][0][0] for i in range(5)]))/4.0
ave_gothrough_hier = np.sum(np.asarray([atthier_acc[i][1][0] for i in range(5)]))/4.0

print ave_concate_hier
print ave_gothrough_hier

79.9191374663
80.1212938005


In [10]:

nonatt_ave_concate_hier = np.sum(np.asarray([nonatthier_acc[i][0][0] for i in range(5)]))/5.0
nonatt_ave_gothrough_hier = np.sum(np.asarray([nonatthier_acc[i][1][0] for i in range(5)]))/5.0

print nonatt_ave_concate_hier
print nonatt_ave_gothrough_hier

80.3234501348
80.4851752022


In [13]:
val_test_pair = account_acc('PlaintEnd')

In [14]:
val_acc_ls = np.asarray([val_test_pair[i][0] for i in range(5)])
ave_acc = sum(val_acc_ls)/ 5.0
print ave_acc

76.4420485175


In [13]:
import cPickle as pickle
import numpy as np

with open('../../data/pickles/test_index_corpus.pkl', 'r') as f:
    test_set = pickle.load(f)
    test_ans = test_set[3]
test_ans_vec = np.asarray(test_ans)

In [17]:

five_scores = []
for i in range(5):
    print str(i)
    f = open('./NonAttHier_record_matrixsequence-gothrough-DNNhinge-150x75x1-1-0.0-'+str(i)+'.pkl', 'r')
    a = pickle.load(f)
    f.close()
    
    five_scores.append(a)
end1_scores = [five_scores[i][:,0] for i in range(5)]
end2_scores = [five_scores[i][:,1] for i in range(5)]

def concate(end_scores):
    end_reshape = [end1.reshape((-1,1)) for end1 in end_scores]
    end_concate = np.concatenate(end_reshape, axis = 1)
    return end_concate

end1_matrix = concate(end1_scores)
end2_matrix = concate(end2_scores)

final_end1_score = end1_matrix.sum(axis = 1).reshape((-1,1))
final_end2_score = end2_matrix.sum(axis = 1).reshape((-1,1))
final_decision = np.argmax(np.concatenate((final_end1_score,final_end2_score), axis = 1),axis = 1)

acc = (1871.0 - np.count_nonzero((final_decision - test_ans_vec)))/ 1871.0
print acc

0
1
2
3
4
0.721539283805


In [86]:
end1_scores = [five_scores[i][:,0] for i in range(5)]
end2_scores = [five_scores[i][:,1] for i in range(5)]

In [87]:
import numpy as np
def concate(end_scores):
    end_reshape = [end1.reshape((-1,1)) for end1 in end_scores]
    end_concate = np.concatenate(end_reshape, axis = 1)
    return end_concate

end1_matrix = concate(end1_scores)
end2_matrix = concate(end2_scores)


In [88]:
final_end1_score = end1_matrix.sum(axis = 1).reshape((-1,1))
final_end2_score = end2_matrix.sum(axis = 1).reshape((-1,1))
final_decision = np.argmax(np.concatenate((final_end1_score,final_end2_score), axis = 1),axis = 1)


In [89]:
with open('../../data/pickles/test_index_corpus.pkl', 'r') as f:
    test_set = pickle.load(f)
    test_ans = test_set[3]
test_ans_vec = np.asarray(test_ans)

In [90]:
acc = (1871.0 - np.count_nonzero((final_decision - test_ans_vec)))/ 1871.0


In [91]:
print acc

0.663281667557


In [54]:
with open('./test_score_record_matrix_plaint_GoThroughlast--hinge-512x1-1-0.2-4.pkl','r') as f:
    print pickle.load(f)

[[ 0.86403882  0.08119898]
 [-0.659258   -0.66733739]
 [ 0.95036546 -0.95985331]
 ..., 
 [-0.61107695  0.91769562]
 [ 0.7564098   0.89060787]
 [ 0.87376921  0.63988742]]
